In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
#from imblearn.under_sampling import RandomUnderSampler
#from skimage.transform import rotate
import cv2

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
img_size=256
batch_size=25

In [4]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=5,
    vertical_flip=True,
    data_format="channels_last",
    validation_split=0.25,
)

In [5]:
train_generator = datagen.flow_from_directory(
    r"/kaggle/input/diabetic-retinopathy-resized-arranged",
    target_size=(img_size, img_size),
    color_mode="grayscale",
    class_mode="sparse",
    shuffle=True,
    subset="training",
)

val_generator = datagen.flow_from_directory(
    r"/kaggle/input/diabetic-retinopathy-resized-arranged",
    target_size=(img_size, img_size),
    color_mode="grayscale",
    class_mode="sparse",
    shuffle=True,
    subset="validation",
)


Found 26346 images belonging to 5 classes.
Found 8780 images belonging to 5 classes.


In [9]:
# configure gpu
gpus = tf.config.list_physical_devices('GPU'); 
if len(gpus) == 1:
    strategy = tf.distribute.OneDeviceStrategy(device='/gpu:0')
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy()
else:
    strategy = tf.distribute.OneDeviceStrategy(device='/cpu:0')

In [10]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [6]:
from tensorflow.keras import regularizers

In [13]:
inputs = keras.Input(shape=(256, 256,1))
x = layers.Conv2D(32, 3, activation="relu", padding="same")(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Dropout(0.25)(x)

x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(5, activation="softmax")(x)

model1 = keras.Model(inputs=inputs, outputs=output)

model1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

model1.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 256, 256, 32)      320       
                                                                 
 batch_normalization_4 (Batc  (None, 256, 256, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 128, 128, 32)      0   

In [14]:
model1.fit(train_generator,batch_size=25,epochs=15, verbose=2)
model1.evaluate(val_generator,batch_size=25, verbose=2)

Epoch 1/15
824/824 - 263s - loss: 3.7861 - accuracy: 0.7237 - 263s/epoch - 319ms/step
Epoch 2/15
824/824 - 256s - loss: 0.9070 - accuracy: 0.7348 - 256s/epoch - 311ms/step
Epoch 3/15
824/824 - 252s - loss: 0.8872 - accuracy: 0.7346 - 252s/epoch - 306ms/step
Epoch 4/15
824/824 - 245s - loss: 0.8770 - accuracy: 0.7348 - 245s/epoch - 297ms/step
Epoch 5/15
824/824 - 247s - loss: 0.8741 - accuracy: 0.7348 - 247s/epoch - 300ms/step
Epoch 6/15
824/824 - 250s - loss: 0.8714 - accuracy: 0.7348 - 250s/epoch - 303ms/step
Epoch 7/15
824/824 - 249s - loss: 0.8697 - accuracy: 0.7347 - 249s/epoch - 303ms/step
Epoch 8/15
824/824 - 252s - loss: 0.8700 - accuracy: 0.7348 - 252s/epoch - 306ms/step
Epoch 9/15
824/824 - 254s - loss: 0.8672 - accuracy: 0.7348 - 254s/epoch - 308ms/step
Epoch 10/15
824/824 - 252s - loss: 0.8694 - accuracy: 0.7347 - 252s/epoch - 306ms/step
Epoch 11/15
824/824 - 250s - loss: 0.8695 - accuracy: 0.7348 - 250s/epoch - 304ms/step
Epoch 12/15
824/824 - 245s - loss: 0.8695 - accuracy

[0.8683276772499084, 0.7348519563674927]

In [16]:
model1.save_weights("/kaggle/working/weights_model1/")

In [17]:
model1.save("/kaggle/working/model1/")

In [17]:
inputs = keras.Input(shape=(256, 256, 1))
x = layers.Conv2D(
    filters=32,
    kernel_size=3,
    padding="same")(inputs)
#x = layers.BatchNormalization()(x)
x = keras.activations.relu(x)
x = layers.Conv2D(32, 3, padding="same")(x)
#x = layers.BatchNormalization()(x)
x = keras.activations.sigmoid(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(128, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Dropout(0.25)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
x = layers.Conv2D(64, 3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Dropout(0.25)(x)

x = layers.Flatten()(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(5, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=output)

model.compile(
    optimizer=keras.optimizers.Adam(0.002),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"])

model.summary()



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 32)      320       
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 256, 256, 32)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 256, 32)      9248      
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 256, 256, 32)     0         
 )                                                               
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 32)     0         
 2D)                                                       

In [19]:
train_generator_1 = datagen.flow_from_directory(
    r"/kaggle/input/diabetic-retinopathy-resized-arranged",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="sparse",
    shuffle=True,
    subset="training",
)

val_generator_1 = datagen.flow_from_directory(
    r"/kaggle/input/diabetic-retinopathy-resized-arranged",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="sparse",
    shuffle=True,
    subset="validation",
)

Found 26346 images belonging to 5 classes.
Found 8780 images belonging to 5 classes.


In [21]:
new_model.fit(train_generator_1,batch_size=25,epochs=10, verbose=2)
new_model.evaluate(val_generator_1,batch_size=25, verbose=2)

/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/10
824/824 - 476s - loss: 0.8393 - accuracy: 0.7322 - 476s/epoch - 578ms/step
Epoch 2/10
824/824 - 469s - loss: 0.7781 - accuracy: 0.7408 - 469s/epoch - 569ms/step
Epoch 3/10
824/824 - 462s - loss: 0.7635 - accuracy: 0.7450 - 462s/epoch - 561ms/step
Epoch 4/10
824/824 - 461s - loss: 0.7487 - accuracy: 0.7480 - 461s/epoch - 560ms/step
Epoch 5/10
824/824 - 473s - loss: 0.7398 - accuracy: 0.7503 - 473s/epoch - 574ms/step
Epoch 6/10
824/824 - 476s - loss: 0.7320 - accuracy: 0.7523 - 476s/epoch - 577ms/step
Epoch 7/10
824/824 - 478s - loss: 0.7270 - accuracy: 0.7534 - 478s/epoch - 580ms/step
Epoch 8/10
824/824 - 475s - loss: 0.7236 - accuracy: 0.7557 - 475s/epoch - 576ms/step
Epoch 9/10
824/824 - 462s - loss: 0.7146 - accuracy: 0.7566 - 462s/epoch - 561ms/step
Epoch 10/10
824/824 - 465s - loss: 0.7134 - accuracy: 0.7580 - 465s/epoch - 565ms/step
275/275 - 200s - loss: 0.7144 - accuracy: 0.7575 - 200s/epoch - 729ms/step


[0.7143973708152771, 0.7575170993804932]

In [2]:
new_model.save_weights("/kaggle/working/weights_transfer_l/")

NameError: name 'new_model' is not defined

In [27]:
y_pred=new_model.predict(val_generator)

22/22 [==============================] - 8s 272ms/step


In [29]:
y_pred.shape

(686, 5)

In [30]:
y_pred

array([[0.7481307 , 0.0406514 , 0.08775906, 0.10397578, 0.01948317],
       [0.01062862, 0.09073132, 0.34643203, 0.3544849 , 0.19772315],
       [0.7452619 , 0.08022813, 0.0657474 , 0.08908273, 0.01967986],
       ...,
       [0.00122486, 0.00679002, 0.27932847, 0.2807906 , 0.43186602],
       [0.832871  , 0.03035722, 0.06618261, 0.05446028, 0.01612891],
       [0.97948235, 0.00568248, 0.00936423, 0.00356527, 0.00190575]],
      dtype=float32)

In [31]:
predictions=tf.argmax(y_pred,1)

In [32]:
predictions

<tf.Tensor: shape=(686,), dtype=int64, numpy=
array([0, 3, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 3, 2, 2, 2,
       0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 0, 3, 0, 0, 2, 0, 2, 2, 0,
       0, 2, 2, 1, 2, 0, 1, 0, 0, 0, 2, 2, 3, 0, 2, 0, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 0, 0, 2, 0, 3, 0, 1, 0, 2, 0, 2, 2, 0, 2, 2, 1, 3, 0,
       1, 3, 3, 2, 2, 0, 2, 1, 0, 0, 0, 0, 2, 1, 2, 2, 3, 2, 0, 2, 2, 0,
       0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 1, 0, 3, 0, 2, 0, 2, 3, 2, 0, 3, 0,
       2, 0, 2, 0, 0, 2, 0, 3, 0, 3, 0, 2, 2, 0, 3, 2, 2, 3, 2, 2, 2, 3,
       0, 2, 1, 2, 2, 2, 2, 2, 3, 0, 2, 3, 0, 0, 2, 2, 0, 2, 3, 1, 0, 0,
       2, 0, 2, 2, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 2, 0, 1, 0, 0, 4, 0, 3, 0, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 0, 2,
       3, 2, 0, 2, 3, 0, 0, 2, 3, 0, 1, 0, 0, 0, 0, 0, 2, 2, 0, 3, 2, 2,
       3, 2, 0, 3, 0, 1, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 3, 0, 0, 3,
       0, 2, 0, 2, 0, 3, 0, 0, 0, 3, 2, 0, 0, 3, 3, 0, 0, 1, 2, 1, 0, 0,
     

In [37]:
for _ in range(5):
    # Retrieve a batch of images and labels from the iterator
    batch_images, batch_labels = next(val_generator_1)

In [39]:
batch_images.shape

(32, 256, 256, 3)

In [41]:
batch_labels

array([1., 2., 2., 0., 3., 0., 0., 2., 2., 4., 1., 0., 0., 2., 0., 0., 0.,
       2., 0., 0., 2., 0., 1., 4., 2., 2., 1., 2., 0., 4., 2., 1.],
      dtype=float32)